In [1]:
import pandas as pd
import numpy as np


from datetime import datetime, date, time, timedelta
import calendar as c         # c is the module
c.setfirstweekday(c.SUNDAY)  # first day Sunday America not Monday Euro
cc = c.Calendar()            # cc is the an instantiation of the class Calendar; class Calendar inside module c


from dateutil.relativedelta import *
from dateutil.rrule import *
from dateutil.parser import *


from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set(style='darkgrid')

from scipy.stats import mannwhitneyu

import copy


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv('../data/df_5.csv')
df_corpus_vile_non = copy.deepcopy(df)             

In [3]:
df_weather = pd.read_csv('../data/noaa_weather_us.csv')      # US Govt O'Hare Airport Station, daily max temps
                                                             # thank-you Uncle Sam

In [4]:
df['date_col'] = pd.to_datetime(df['date_col'])    
df['date'] = pd.to_datetime(df['date'])            
df.index = df['date']        
df = df.drop(['date'], axis= 1)

In [5]:
df.head(1)

,date_col,id,case_number,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,year,updated_on,latitude,longitude,location,district_x,sector,beat_x,beat_num,district_name,weekday,fri_sat_sun,hour_of_day,summation_bit,first_of_month,first_week_from_first,second_week_from_first,last_week_from_end,second_to last_week_from_end,month,year_month,street
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-11-15 00:01:00,2012-11-15 00:01:00,8926266,HV598372,013XX W CORNELIA AVE,0810,THEFT,OVER $500,APARTMENT,False,False,1924,19.0,44.0,6.0,06,2012,02/10/2018 03:50:01 PM,41.945267,-87.662672,"(41.945267201, -87.662671562)",19.0,2.0,2.0,1924.0,Town Hall,Thursday,0,0,1,0,0,0,0,0,11,2012-11,CORNELIAAVE


In [6]:
df_weather.head()

,STATION,DATE,MAX
0,72530094846,2012-11-15,48.0
1,72530094846,2012-11-16,54.0
2,72530094846,2012-11-17,55.0
3,72530094846,2012-11-18,57.0
4,72530094846,2012-11-19,59.0


In [7]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 3 columns):
STATION    2557 non-null int64
DATE       2557 non-null object
MAX        2557 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 60.1+ KB


In [8]:
df_weather.columns = ['station', 'date_col', 'max']

In [9]:
temp_tops = [i for i in range(65, 105, 5)]
for i in temp_tops:
    print('max temp {} : {} '.format(i, df_weather[df_weather['max'] >= i].shape[0]))

max temp 65 : 1227 
max temp 70 : 1109 
max temp 75 : 941 
max temp 80 : 730 
max temp 85 : 353 
max temp 90 : 150 
max temp 95 : 23 
max temp 100 : 0 


In [10]:
df_daily_crime = df.resample('D').size().to_frame()
df_daily_crime.columns = ['event_count']

In [11]:
df_daily_crime.head()

,event_count
date,
2012-11-15,936
2012-11-16,941
2012-11-17,874
2012-11-18,849
2012-11-19,866


In [12]:
# de-foobar the index messes that arise from groupbys/resamples to get a legit-to-use date_col for outer merge
constructor_list_1 = []
for item in list(df_daily_crime.index):
    constructor_list_1.append(str(item))
    
constructor_list_2 = []
for item in constructor_list_1:
    constructor_list_2.append(''.join(item.split()[0]))

In [13]:
df_daily_crime['date_col'] = constructor_list_2

In [14]:
df_daily_crime.head(1)

,event_count,date_col
date,,
2012-11-15,936,2012-11-15


In [15]:
df_weather.head(1)

,station,date_col,max
0,72530094846,2012-11-15,48.0


In [16]:
df_combined = pd.merge(df_daily_crime, df_weather, how= 'outer', on= 'date_col')

In [17]:
df_combined.isnull().sum()

event_count    2
date_col       0
station        4
max            4
dtype: int64

In [18]:
df_combined = df_combined.dropna()

In [19]:
def flag_jun_july_aug(row):
    event_month = int(row['date_col'].split('-')[1])
    return int((event_month == 6) | (event_month == 7) | (event_month == 8))

In [20]:
df_combined['j_j_a_flag'] = df_combined.apply(flag_jun_july_aug, axis= 1)

In [21]:
df_85 = df_combined[(df_combined['max'] >= 85) & (df_combined['j_j_a_flag'] == 1)]
df_90 = df_combined[(df_combined['max'] >= 90) & (df_combined['j_j_a_flag'] == 1)]

In [22]:
df_85.shape

(272, 5)

In [23]:
df_90.shape

(118, 5)

In [24]:
df_84 = df_combined[(df_combined['max'] < 85) & (df_combined['j_j_a_flag'] == 1)]
df_89 = df_combined[(df_combined['max'] < 90) & (df_combined['j_j_a_flag'] == 1)]

In [25]:
df_84.shape

(372, 5)

In [26]:
df_89.shape

(526, 5)

##### The groups are unbalanced so cannot use t-test. Use Mann Whitney test instead again.

##### The scipy stats doc says valid for samples n >= 20 or above.   Unbalanced groups are fine but each group must be at least 20.

##### n_above_85 == 272 & n_below_85 == 372    Requiements met fire away,

## Null Hypothesis

##### My Null Hypothesis is that there is no difference in the average number of crimes on the days over 85 degrees in the summer months of June July and August than the average number of crimes when its below 85 degrees in the summer months of June July August

In [27]:
#     H_0 :   mu_summer_over_85 == mu_summer_under_85

In [28]:
#     Null is that the both both temperature crime samples are coming from a single population.

In [29]:
list_85 = list(df_85['max'])
list_84 = list(df_84['max'])

In [30]:
stat, p = mannwhitneyu(list_85, list_84)

In [31]:
p

6.1245926637662694e-105

## During the Summer months of JJA,  the temperature over 85 degrees does have an effect on average crime rates in Chicago.